In [1]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq


In [ ]:
import os
llm = Groq(model="llama-3.3-70b-versatile",
           api_key= os.environ.get("GROQ_API_KEY"))

In [3]:
def calcular_imposto_renda(rendimento: float) -> str:
    """
    Calcula o imposto de renda com base no rendimento anual.
    
    Args:
        rendimento(float): Rendimento anual do indivíduo.
        
    Returns:
        str
    """
    if rendimento <= 2000:
        return "Você está isento de pagar imposto de renda."
    elif 2000 < rendimento <= 5000:
        imposto = (rendimento - 2000) * 0.10
        return f"Você deve pagar R$ {imposto:.2f} de imposto de renda, com base no seu rendimento de R$ {rendimento:.2f}."
    elif 5000 < rendimento <= 10000:
        imposto = (rendimento - 5000) * 0.15 + 300
        return f"Você deve pagar R$ {imposto:.2f} de imposto de renda, com base no seu rendimento de R$ {rendimento:.2f}."
    else:
        imposto = (rendimento - 10000) * 0.20 + 1200
        return f"Você deve pagar R$ {imposto:.2f} de imposto de renda, com base no seu rendimento de R$ {rendimento:.2f}."

### Convertendo função em ferramenta

In [4]:
from llama_index.core.tools import FunctionTool

In [ ]:
#Criando ferramenta

ferramenta_imposto_renda = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name= "CAlcular imposto de renda",
    description=(
        "Calcula o imposto de renda com base no rendimento anual."
        "Argumento: rendimento (float)."
        "Retorna o valor do imposto devido de acordo com as faixas de rendimento."
    )
    
)

In [6]:
from llama_index.core.agent import FunctionCallingAgentWorker

In [7]:
agent_worker_imposto= FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta_imposto_renda],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm)

In [8]:
from llama_index.core.agent import AgentRunner

In [9]:
agent_imposto = AgentRunner(agent_worker_imposto)

In [10]:
response = agent_imposto.chat("""
                              Qalcular o imposto de renda para um rendimento anual de R$ 7500.00.""")

Added user message to memory: 
                              Qalcular o imposto de renda para um rendimento anual de R$ 7500.00.
=== Calling Function ===
Calling function: CAlcular imposto de renda with args: {"rendimento": 7500}
=== Function Output ===
Você deve pagar R$ 675.00 de imposto de renda, com base no seu rendimento de R$ 7500.00.
=== Calling Function ===
Calling function: CAlcular imposto de renda with args: {"rendimento": 7500}
=== Function Output ===
Você deve pagar R$ 675.00 de imposto de renda, com base no seu rendimento de R$ 7500.00.
=== Calling Function ===
Calling function: CAlcular imposto de renda with args: {"rendimento": 7500}
=== Function Output ===
Você deve pagar R$ 675.00 de imposto de renda, com base no seu rendimento de R$ 7500.00.
=== Calling Function ===
Calling function: CAlcular imposto de renda with args: {"rendimento": 7500}
=== Function Output ===
Você deve pagar R$ 675.00 de imposto de renda, com base no seu rendimento de R$ 7500.00.
=== Calling Fun

In [11]:
response = agent_imposto.chat("Quantos anos tem 1000 dias?")

Added user message to memory: Quantos anos tem 1000 dias?
=== LLM Response ===
1000 dias equivalem a aproximadamente 2,74 anos.


In [12]:
import arxiv # biblioteca de artigos científicos

def consulta_artigos(title: str)-> str:
    """Consulta os arquivos na base de dados ArXiv e retorna resultafos FOrmatados"""
    busca = arxiv.Search(
        query=title,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance     
    )
    
    resultados= [
        f"Título: {artigo.title}\n"
        f"Categoria:{artigo.primary_category}\n"
        f"Link: {artigo.entry_id}\n"
        for artigo in busca.results()
    ]
    
    return "\n\n".join(resultados)

In [14]:
consilta_artigos_tool = FunctionTool.from_defaults(
    fn = consulta_artigos,
    name = "Consulta artigos científicos",
    description=(
        "Consulta artigos científicos na base de dados ArXiv."
        "Argumento: title (str)."
        "Retorna os títulos, categorias e links dos artigos encontrados."
    )
)

In [18]:
#criando agente com a ferramenta de consulta de artigos
from llama_index.core.agent import FunctionCallingAgentWorker
agent_worker = FunctionCallingAgentWorker.from_tools(
    [consilta_artigos_tool,ferramenta_imposto_renda],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm)


In [19]:
agent = AgentRunner(agent_worker)
response = agent.chat("Quais artigos científicos falam sobre inteligência artificial?")

Added user message to memory: Quais artigos científicos falam sobre inteligência artificial?
=== Calling Function ===
Calling function: Consulta artigos científicos with args: {"title": "intelig\u00eancia artificial"}


C:\Users\NYLUNA\AppData\Local\Temp\ipykernel_10056\2503167639.py:15: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for artigo in busca.results()


=== Function Output ===
Título: Does an artificial intelligence perform market manipulation with its own discretion? -- A genetic algorithm learns in an artificial market simulation
Categoria:q-fin.TR
Link: http://arxiv.org/abs/2005.10488v1


Título: The Governance of Physical Artificial Intelligence
Categoria:cs.AI
Link: http://arxiv.org/abs/2304.02924v1


Título: Perspective: Purposeful Failure in Artificial Life and Artificial Intelligence
Categoria:cs.AI
Link: http://arxiv.org/abs/2102.12076v1


Título: Taking the redpill: Artificial Evolution in native x86 systems
Categoria:cs.NE
Link: http://arxiv.org/abs/1105.1534v1


Título: The case for psychometric artificial general intelligence
Categoria:cs.AI
Link: http://arxiv.org/abs/2101.02179v1

=== LLM Response ===
Esses artigos científicos falam sobre inteligência artificial.


In [22]:
response = agent.chat("oi")

Added user message to memory: oi
=== LLM Response ===
Olá! Como posso ajudar você hoje?


## Usando Tavily 

In [33]:
import os
from dotenv import load_dotenv,find_dotenv

_ = load_dotenv(find_dotenv())

In [34]:
tavily_key = api_key= os.environ.get("TAVILY_API_KEY")

In [35]:
from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key= tavily_key
)

In [36]:
tavily_tool_list = tavily_tool.to_tool_list()
for tool in tavily_tool_list:
    print(tool.metadata.name)

search


In [37]:
tavily_tool.search("ME retorne artigos científicos sobre LangChain", max_results= 3)

[Document(id_='57a612f5-bd5b-4723-823b-971c8c7ed00d', embedding=None, metadata={'url': 'https://python.langchain.com/docs/additional_resources/arxiv_references/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Code Generation with AlphaCodium: From Prompt Engineering to Flow Engineering\u200b\n--------------------------------------------------------------------------------------------------------------------------------\n\n   Authors: Tal Ridnik, Dedy Kredo, Itamar Friedman\n    \n   arXiv id: 2401.08500v1 Published Date: 2024-01-16\n    \n   LangChain:\n    \n       Documentation: docs/concepts [...] Corrective Retrieval Augmented Generation\u200b\n--------------------------------------------------------------------------------------------------------------------------------\n\n   Authors: Shi-Qi Yan, Jia-Chen Gu, Yun Zhu, et al

 Criando ferramenta com uma ferramenta ja criada no caso o tavily


In [39]:
from llama_index.core.tools import FunctionTool

tavily_tool_function = FunctionTool.from_defaults(
    fn=tavily_tool.search,
    name="Tavily Search",
    description=("Busca artigos com o Tavily sobre um determinado tópico")
)

Criando o agente

In [40]:
agent_worker_tavily = FunctionCallingAgentWorker.from_tools(
    tools=[tavily_tool_function],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [41]:
agent = AgentRunner(agent_worker_tavily)

In [42]:
response = agent.chat("Quais artigos científicos falam sobre LangChain?")

Added user message to memory: Quais artigos científicos falam sobre LangChain?
=== Calling Function ===
Calling function: Tavily Search with args: {"max_results": 10, "query": "LangChain"}
=== Function Output ===
[Document(id_='57568ef2-de35-4252-9743-84f110c98813', embedding=None, metadata={'url': 'https://en.wikipedia.org/wiki/LangChain'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Image 5Free and open-source software portal\n\nLangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.(\n\nHistory\n-------\n\n[edit] [...] LangChain was launched in October 2022 as an

In [43]:
print(response)

Os artigos científicos que falam sobre LangChain incluem:

1. "LangChain: A Framework for Building LLM-Powered Applications" - Este artigo apresenta uma visão geral do LangChain, um framework de código aberto para desenvolver aplicações impulsionadas por grandes modelos de linguagem (LLMs).
2. "LangChain: A Tool for Building AI-Powered Applications" - Este artigo discute como o LangChain pode ser usado para criar aplicações de IA que integram modelos de linguagem com ferramentas e APIs externas.
3. "Introdução ao LangChain: Um Framework para Desenvolver Aplicações de IA" - Este artigo fornece uma introdução ao LangChain e explica como ele pode ser usado para desenvolver aplicações de IA que utilizam modelos de linguagem.
4. "LangChain: Uma Ferramenta para Construir Aplicações de IA com Modelos de Linguagem" - Este artigo apresenta uma visão geral do LangChain e explica como ele pode ser usado para criar aplicações de IA que utilizam modelos de linguagem.
5. "Desenvolvendo Aplicações de

## Genrenciando Embeddings e Engine de Busca

In [44]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

In [46]:
url = "files/LLM.pdf"
artigo = SimpleDirectoryReader(input_files=[url]).load_data()

Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 158 0 (offset 0)
Ignoring wrong pointing object 160 0 (offset 0)
Ignoring wrong pointing object 163 0 (offset 0)
Ignori

In [48]:
url = "files/LLM_2.pdf"
tutorial = SimpleDirectoryReader(input_files=[url]).load_data()

## Gerar os Embeddings com hunggingface

In [50]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [51]:
Settings.default_embedding = HuggingFaceEmbedding(
    model_name = "intfloat/multilingual-e5-large"
    
)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

c:\Users\NYLUNA\OneDrive - Deloitte (O365D)\Desktop\LLamaIndex_Agents\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\NYLUNA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\Local\llama_index\models--intfloat--multilingual-e5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warn

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [54]:
artigo_index = VectorStoreIndex.from_documents(
	artigo,
	embed_model=Settings.default_embedding
)
tutorial_index = VectorStoreIndex.from_documents(
	tutorial,
	embed_model=Settings.default_embedding
)

In [ ]:
# Persistindo os índices
# Isso salva os índices em um diretório específico para uso posterior
artigo_index.storage_context.persist(
    persist_dir="artigo")
tutorial_index.storage_context.persist(
    persist_dir="tutorial")

### Engine de Busca

In [56]:
from llama_index.core import StorageContext, load_index_from_storage

In [59]:
storage_context  = StorageContext.from_defaults(
    persist_dir="artigo"
)
artigo_index = load_index_from_storage(
    storage_context=storage_context,
    embed_model=Settings.default_embedding
)
storage_context  = StorageContext.from_defaults(
    persist_dir="tutorial"
)
tutorial_index = load_index_from_storage(
    storage_context=storage_context,
    embed_model=Settings.default_embedding
)

In [60]:
artigo_engine = artigo_index.as_query_engine(
    similarity_top_k=3, llm=llm)
tutorial_engine = tutorial_index.as_query_engine(
    similarity_top_k=3, llm=llm)

In [61]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [62]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=artigo_engine,
        metadata=ToolMetadata(
            name="Artigo Engine",
            description=("Fornece informações sobre LLM e LAngChain "
                         "Use uma pergunta detalhada em texto simples como entrada para a ferramenta.")
        )
    ),
    QueryEngineTool(
        query_engine=tutorial_engine,
        metadata=ToolMetadata(
            name="Tutorial Engine",
            description=("Fornece informações sobre casos de uso e aplicações em LLM"
                         "Use uma pergunta detalhada em texto simples como entrada para a ferramenta.")
        )
    )
    
]

In [63]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

agent_document = AgentRunner(agent_worker)

In [71]:
response = agent_document.chat("Quais as principais aplicações posso construir com LLMs e LangChain e langgraph?")

Added user message to memory: Quais as principais aplicações posso construir com LLMs e LangChain e langgraph?
=== Calling Function ===
Calling function: Tutorial Engine with args: {"input": "Quais as principais aplica\u00e7\u00f5es posso construir com LLMs e LangChain e LangGraph?"}
=== Function Output ===
As principais aplicações que você pode construir com LLMs incluem:

1. **Criação e aprimoramento de conteúdo**: como geração automática de texto, assistência na redação, tradução automática, resumo de textos, planejamento e roteiro de conteúdo, e brainstorming.
2. **Análise e organização de informações**: como análise de sentimento, extração de informações, classificação de textos, e revisão técnica.
3. **Interação e automação**: como chatbots, perguntas e respostas, e geração de respostas a perguntas com base em um corpus.

Além disso, com a integração de LLMs a ferramentas de desenvolvimento de software e de escritório, é possível construir aplicações avançadas em diversos setores

In [70]:
# Exibir as fontes da resposta, se existirem
print(response.sources)
# Persistindo os índices    

[ToolOutput(content='As principais aplicações que você pode construir com LLMs incluem:\n\n1. Criação e aprimoramento de conteúdo, como geração automática de texto, assistência na redação, tradução automática, resumo de textos e planejamento de conteúdo.\n2. Análise e organização de informações, como análise de sentimento, extração de informações, classificação de textos e revisão técnica.\n3. Interação e automação, como chatbots, perguntas e respostas, e geração de respostas a perguntas com base em um corpus.\n\nAlém disso, com o surgimento dos LLMs multimodais, outras aplicações estão começando a surgir, como a geração de conteúdo audiovisual, a interpretação de dados de imagens, a tradução de conteúdo multimídia e a criação de experiências interativas ricas.\n\nJá com relação às aplicações práticas em produção, exemplos incluem:\n\n* Chatbots internos para facilitar o acesso a informações da empresa\n* Extração de informações de documentos grandes e complexos\n* Suporte ao centro de

In [72]:
response = agent_document.chat("Quais as princeipais tendencias em langchain")

Added user message to memory: Quais as princeipais tendencias em langchain
=== Calling Function ===
Calling function: Tutorial Engine with args: {"input": "Quais as principais tend\u00eancias em LangChain?"}
=== Function Output ===
As principais tendências em LangChain incluem a proliferação de LLMs de código aberto, que democratizou o acesso à tecnologia de ponta de processamento de linguagem, permitindo que pesquisadores, desenvolvedores e amadores experimentassem, personalizassem e implantassem soluções de IA com um investimento inicial mínimo. Além disso, a integração do LLM às ferramentas de desenvolvimento de software e de escritório está transformando a eficiência e a capacidade das empresas. Outra tendência é a evolução dos LLMs para além da simples previsão de texto, tornando-se aplicativos sofisticados em vários domínios, arquiteturas e modalidades, com categorização dos LLMs de acordo com vários critérios, como arquitetura e componente.
=== Calling Function ===
Calling funct

### Agente React

In [73]:
from llama_index.core.agent import ReActAgent

In [78]:
agent = ReActAgent.from_tools(
    tools=query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm,
    max_iterations=30 )

In [80]:
import time

while True:
	try:
		response = agent.chat("Quais as princeipais tendencias em langchain")
		break
	except Exception as e:
		if "rate limit" in str(e).lower() or "429" in str(e):
			print("Rate limit reached. Waiting for 4 minutes before retrying...")
			time.sleep(240)  # Wait for 4 minutes
		else:
			raise e

> Running step c96b9115-81b0-458d-8650-593f89d3dbb3. Step input: Quais as princeipais tendencias em langchain


Retrying llama_index.llms.openai.base.OpenAI._chat in 1.0 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jhr099anev18ja2fvgqm54dr` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99482, Requested 766. Please try again in 3m33.938s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}.


Thought: O usuário está perguntando sobre as principais tendências em LangChain. Para responder a essa pergunta, preciso usar uma ferramenta que forneça informações sobre LangChain.
Action: Artigo
Action Input: {'input': 'principais tendências em LangChain'}
Observation: Error: No such tool named `Artigo`.
> Running step a2d1945c-2cf1-4614-aa21-27b124736c12. Step input: None


Retrying llama_index.llms.openai.base.OpenAI._chat in 1.828445436695826 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jhr099anev18ja2fvgqm54dr` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99481, Requested 766. Please try again in 3m32.858s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}.


Rate limit reached. Waiting for 4 minutes before retrying...
> Running step fae43dcc-ca9a-4e46-8361-fe2cd28281ac. Step input: Quais as princeipais tendencias em langchain


...

Retrying llama_index.llms.openai.base.OpenAI._chat in 1.5902989267926364 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jhr099anev18ja2fvgqm54dr` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99909, Requested 759. Please try again in 9m36.828s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}.


Rate limit reached. Waiting for 4 minutes before retrying...
> Running step 83907b21-0d20-44ed-a2e9-27d4cad51b0b. Step input: Quais as princeipais tendencias em langchain


Retrying llama_index.llms.openai.base.OpenAI._chat in 1.0 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jhr099anev18ja2fvgqm54dr` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99629, Requested 678. Please try again in 4m24.528s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._chat in 1.0821298388500167 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jhr099anev18ja2fvgqm54dr` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99627, Requested 678. Please try again in 4m23.435999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'cod

Rate limit reached. Waiting for 4 minutes before retrying...


KeyboardInterrupt: 